In [ ]:
from __init__ import *

In [ ]:
feature_x_sample = pd.read_csv("../input/feature_x_sample.tsv", sep="\t", index_col=0)

comparison_x_sample = pd.read_csv(
    "../output/comparison_x_sample.tsv", sep="\t", index_col=0
)

In [ ]:
def get_log_ratio(groups, vector):

    assert 0 <= vector.min()

    vector = np.log2(1 + vector)

    return vector[groups == 1].mean() - vector[groups == 0].mean()

In [ ]:
keyword_arguments = {
    "se_data_type": "binary",
    "plot_std": SETTING["plot_std"],
}

for comparison, groups in comparison_x_sample.iterrows():

    groups.dropna(inplace=True)

    min_group_size = np.unique(groups.to_numpy(), return_counts=True)[1].min()

    for function, group_size in (
        (get_log_ratio, 1),
        (kraft.information.get_ic, 2),
    ):

        if min_group_size < group_size:

            continue

        directory_path = "../output/compare_feature/{}/{}/".format(
            comparison, function.__name__
        )

        kraft.path.path(directory_path)

        scores = kraft.function_heat_map.make(
            groups,
            feature_x_sample,
            function,
            n_job=SETTING["n_job"],
            n_sampling=SETTING["n_sampling"],
            n_permutation=SETTING["n_permutation"],
            n_extreme=SETTING["n_extreme"],
            title="All ({})".format(function.__name__),
            directory_path=directory_path,
            **keyword_arguments,
        )

        if 0 < len(SETTING["features_to_peek"]):

            kraft.function_heat_map.make(
                groups,
                feature_x_sample.reindex(labels=SETTING["features_to_peek"]),
                scores,
                n_extreme=None,
                title="Peek ({})".format(function.__name__),
                **keyword_arguments,
            )